In [68]:
import pandas as pd
import numpy as np
from decimal import Decimal

x_chat = chatGPT()
x_gemini = gemini()




In [70]:
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
# df_exp = df_exp[df_exp.Title=="Fundraising Managers"]
df_exp.Embeddings = df_exp.Embeddings.apply(lambda x: x.strip("[]").split(", "))
embeddings = []
for x in df_exp.Embeddings:
    embeddings.append([float(y) for y in x])


embeddings = np.array(embeddings)
df_exp.Embeddings = embeddings
embeddings = np.vstack(embeddings)
embeddings.shape


In [71]:
embeddings.shape

(11582, 3072)

In [73]:
sample_df = pd.read_csv('../output/df_with_examples_embeddings.csv')
sample_df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

# list(sample_df.Job1_embedding.apply(ast.literal_eval))
for i in range(3):
    sample_df[f"Job{i+1}_embedding"] = sample_df[f"Job{i+1}_embedding"].apply(lambda x: [Decimal(y) for y in x.strip("[]").split(", ")])
    sample_df[f"Job{i+1}_title_embedding"] = sample_df[f"Job{i+1}_title_embedding"].apply(lambda x: [Decimal(y) for y in x.strip("[]").split(", ")])




In [ ]:

results = []
for index, row in list(sample_df.iterrows())[0:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(3): 
        example = row[f"Example{i+1}"]
        job = row[f'Job{i+1}']
        job_title = row[f'Job{i+1}_title']
        print(f"\nProcessing Example: {example} \nJob: {job}\n")
        job_embedding = np.array(row[f'Job{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Job{i+1}_title_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, embeddings, 'cosine')
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['Title', 'Task']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            onet_title_embedding = np.array([x_chat.get_embedding(onet[0])])
            cosine_sim_title = cosine_similarity(job_title_embedding, onet_title_embedding, 'cosine')[0][0]
            print(f"{onet[0]}: {onet[1]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 5 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.to_csv("../output/onet_df.csv", index=False)
        
    print("***********************\n\n")


***********************
Processing 0
Catio
Catio is a cloud architecture platform that helps companies design and optimize their cloud-native tech stacks. Using AI, Catio provides guidance on technology selection, planning, and continuous improvement, acting as a "copilot" for architects and engineers.

Processing Example: A cloud architect uses Catio to evaluate different cloud service offerings from various providers, comparing costs, performance, and security features to select the most suitable options for their company's specific needs. 
Job: Computer and Information Systems Managers that evaluate cloud service offerings from different providers to select the most suitable options for the company's needs.

Computer and Information Systems Managers: Evaluate the organization's technology use and needs and recommend improvements, such as hardware and software upgrades. 
Cosine Similarity: 0.7308420357826921, Person Cosine Similarity: 0.9999987394994386
Computer and Information Syste